In [6]:
import datetime as dt
import pandas as pd
import numpy as np
import pirep as pr

In [ ]:
# Retrieve PIREPs
raw_reports = pr.fetch(
    pr.url(
        date_s=dt.datetime(2024, 11, 6, 23, 55, 0, tzinfo=dt.timezone.utc),
        date_e=dt.datetime(2024, 11, 7, 0, 5, 0, tzinfo=dt.timezone.utc),
    )
)
raw_reports

In [ ]:
# Retrieve PIREPs
reports = pr.parse_all(raw_reports)
reports = reports.dropna()
reports

In [9]:
from pirep.defs.turbulence import Turbulence
from pirep import compute_grid
from pirep.consts import TURBULENCE_INDEXES
def turbulence_score(x):
    print(x, flush=True)
    turbulence, aircraft = x
    if turbulence.intensity != Turbulence.Intensity.EXT:
        turbulence_index = TURBULENCE_INDEXES[aircraft][turbulence.intensity]
    else:
        turbulence_index = TURBULENCE_INDEXES[aircraft][Turbulence.Intensity.SEV]

    return turbulence_index

xs = reports["Location"].map(lambda x: x.lon)
ys = reports["Location"].map(lambda x: x.lat)
zs = reports["Altitude"].map(lambda x: (x.min // 500) if x.min is not None else 0)
cs = reports.apply(compute_grid, axis=1).to_numpy()

In [10]:
# grouped_grid = np.max(np.stack(grids["Grid"]), axis=0)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl

fig = plt.figure()
ax = fig.add_subplot()
plot = ax.scatter(xs, ys, c=cs, cmap="hot")
ax.set_xlabel("Longitude")
ax.set_ylabel("Latitude")
fig.colorbar(plot)
fig.show()